In [9]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics
import tensorflow.keras as keras

In [2]:
def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = tf.reshape(x, [28*28])
    y = tf.cast(y, dtype=tf.int32)
    y = tf.one_hot(y, depth=10)
    return x,y

In [4]:
batch_size = 32
(x, y), (x_test, y_test) = datasets.mnist.load_data()  # 得到训练集和测试集
# 认为划分数据集
x_train, x_val = tf.split(x, num_or_size_splits=[50000, 10000])
y_train, y_val = tf.split(y, num_or_size_splits=[50000, 10000])

db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
db_train = db_train.map(preprocess).shuffle(50000).batch(batch_size)

db_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
db_val = db_val.map(preprocess).shuffle(10000).batch(batch_size)

db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
db_test = db_test.map(preprocess).batch(batch_size)  # 一般不会打乱测试数据的顺序

In [5]:
network = Sequential([layers.Dense(256, activation='relu'),
                     layers.Dense(128, activation='relu'),
                     layers.Dense(64, activation='relu'),
                     layers.Dense(32, activation='relu'),
                     layers.Dense(10)])
network.build(input_shape=(None, 28*28))
network.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  200960    
_________________________________________________________________
dense_1 (Dense)              multiple                  32896     
_________________________________________________________________
dense_2 (Dense)              multiple                  8256      
_________________________________________________________________
dense_3 (Dense)              multiple                  2080      
_________________________________________________________________
dense_4 (Dense)              multiple                  330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


In [8]:
network.compile(optimizer=optimizers.Adam(lr=0.01), 
                loss=tf.losses.CategoricalCrossentropy(from_logits=True), 
                metrics=['accuracy'])
network.fit(db_train, epochs=5, validation_data=db_val, validation_freq=2)
network.evaluate(db_test)

Epoch 1/5
1563/1563 [==============================] - 18s 12ms/step - loss: 0.1850 - accuracy: 0.9618
Epoch 2/5
1563/1563 [==============================] - 16s 10ms/step - loss: 0.1648 - accuracy: 0.9644 - val_loss: 0.1869 - val_accuracy: 0.9630
Epoch 3/5
1563/1563 [==============================] - 12s 7ms/step - loss: 0.1513 - accuracy: 0.9676
Epoch 4/5
1563/1563 [==============================] - 13s 8ms/step - loss: 0.1557 - accuracy: 0.9678 - val_loss: 0.1687 - val_accuracy: 0.9634
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 0.1955 - accuracy: 0.9559


[0.19548417586917297, 0.9559]

In [ ]:
for epoch in range(500):
    idx = tf.random.shuffle(tf.range(60000))
    x_train, y_train = tf.gather(x, idx[:50000]), tf.gather(y, id[:50000])
    x_val, y_val = tf.gather(x, idx[-10000:]), tf.gather(y, idx[-10000:])
    
    db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    db_train = db_train.map(preprocess).shuffle(50000).batch(batch_size)

    db_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    db_val = db_val.map(preprocess).shuffle(10000).batch(batch_size)
    # training
    
    # validate


In [ ]:
network.fit(db_all, epochs=500, validation_split=0.1, validation_freq=2)

In [ ]:
l2_model = keras.models.Sequential([
    keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.001), 
                       activation=tf.nn.relu, input_shape=(num_words, )),
    keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.001), 
                       activation=tf.nn.relu, input_shape=(num_words, )),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

In [ ]:
for step, (x,y) in enumerate(db):

    with tf.GradientTape() as tape:
        # [b, 28, 28] => [b, 784]
        x = tf.reshape(x, (-1, 28*28))
        # [b, 784] => [b, 10]
        out = network(x)
        # [b] => [b, 10]
        y_onehot = tf.one_hot(y, depth=10) 
        # [b]
        loss = tf.reduce_mean(tf.losses.categorical_crossentropy(y_onehot, out, from_logits=True))


        loss_regularization = []
        for p in network.trainable_variables:
            loss_regularization.append(tf.nn.l2_loss(p))
        loss_regularization = tf.reduce_sum(tf.stack(loss_regularization))

        loss = loss + 0.0001 * loss_regularization
 

    grads = tape.gradient(loss, network.trainable_variables)
    optimizer.apply_gradients(zip(grads, network.trainable_variables))

In [ ]:
rl = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', patience=5)

In [ ]:
es = rl = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5)

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.02, momentum=0.9)